In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import random

# Перевірка версії та наявності GPU (опціонально)
print(f"TensorFlow version: {tf.__version__}")


In [ ]:
# Завантаження даних
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# Нормалізація даних (масштабування до 0-1)
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Зміна розмірності для CNN (додаємо канал кольору)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

print(f"Розмір тренувальних даних: {x_train.shape}")
print(f"Кількість тренувальних зразків: {x_train.shape[0]}")
print(f"Кількість тестових зразків: {x_test.shape[0]}")

# Назви класів
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i].reshape(28,28), cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

In [ ]:
model = keras.Sequential([
    # Перший блок згорток
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25), # Регуляризація

    # Другий блок згорток
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25), # Регуляризація

    # Повнозв'язна частина (Classifier)
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # Сильніша регуляризація перед виходом
    layers.Dense(10, activation='softmax') # 10 класів на виході
])

# Вивід структури моделі
model.summary()

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 20  # Кількість епох (можна спробувати 15-30)
batch_size = 64 # Розмір батчу

history = model.fit(
    x_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2, # 20% даних йде на валідацію
    verbose=1
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 4))

# Графік точності
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Графік втрат
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nТочність на тестових даних: {test_acc*100:.2f}%')

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

def get_label_id(label):
    label = np.array(label)
    label = np.squeeze(label)
    if label.ndim > 0:
        return np.argmax(label)
    return int(label)

def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img.reshape(28,28), cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    true_label_id = get_label_id(true_label)

    if predicted_label == true_label_id:
        color = 'blue'
    else:
        color = 'red'

    # Перевіряємо передбачений клас
    if predicted_label < len(class_names):
        pred_name = class_names[predicted_label]
    else:
        pred_name = f"Unknown({predicted_label})"

    # Перевіряємо справжній клас
    if true_label_id < len(class_names):
        true_name = class_names[true_label_id]
    else:
        true_name = f"Unknown({true_label_id})"

    plt.xlabel("{} {:2.0f}% ({})".format(pred_name,
                                100*np.max(predictions_array),
                                true_name),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    true_label_id = get_label_id(true_label[i])
    
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    
    thisplot = plt.bar(range(10), predictions_array[:10], color="#777777")
    plt.ylim([0, 1])
    
    predicted_label = np.argmax(predictions_array)

    # Захист від виходу за межі масиву при фарбуванні стовпчиків
    if predicted_label < 10:
        thisplot[predicted_label].set_color('red')
    
    if true_label_id < 10:
        thisplot[true_label_id].set_color('blue')

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, y_test, x_test)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], y_test)

plt.tight_layout()
plt.show()